In [1]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
import pandas as pd
import os

In [2]:
sc = SparkSession.builder.appName("PysparkLab3") \
    .config ("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.maxResultSize","5g") \
    .config ("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()

In [3]:
posts_data_dir = r"G:\Data\spark_labs\bigdata20\bigdata20\followers_posts_api_final.json"
group_posts_data_dir = r"G:\Data\spark_labs\bigdata20\bigdata20\posts_api.json"
result_dir = r"G:\Data\spark_labs\bigdata20\bigdata20\results\task3"

posts1_path = os.path.join(posts_data_dir, '0_39773a62_followers_posts_api_final.json')
posts2_path = os.path.join(posts_data_dir, '1_7a8a2098_followers_posts_api_final.json')
group_posts_path = os.path.join(group_posts_data_dir, '0_a99c9b7e_posts_api.json')
posts1_df = sc.read.json(posts1_path)
posts2_df = sc.read.json(posts2_path)
group_posts_df = sc.read.json(group_posts_path)

In [ ]:
group_posts_df.show(10)

In [ ]:
posts1_df.show()

In [ ]:
posts1_df.createOrReplaceTempView("df1_view")
posts2_df.createOrReplaceTempView("df2_view")
group_posts_df.createOrReplaceTempView("group_posts_view")
posts1 = sc.sql("""SELECT id, copy_history, owner_id
                            FROM df1_view 
                            WHERE copy_history IS NOT NULL 
                       """)
posts2 = sc.sql("""SELECT id, copy_history, owner_id
                            FROM df2_view 
                            WHERE copy_history IS NOT NULL 
                       """)

In [51]:
posts1.createOrReplaceTempView("posts1_view")
posts2.createOrReplaceTempView("posts2_view")
reposts_from_group = sc.sql("""SELECT gpv.id, p1v.owner_id 
                            FROM posts1_view p1v, group_posts_view gpv
                            WHERE p1v.copy_history[0].id == gpv.id
                       """)

In [53]:
list_of_reposters = reposts_from_group.groupby("id").agg(collect_list("owner_id"))
list_of_reposters.show()
list_of_reposters.toPandas().to_json(os.path.join(result_dir, 'reposters_list.json'))

+---+----------------------+
| id|collect_list(owner_id)|
+---+----------------------+
| 18|  [110949250, 14793...|
| 26|  [110949250, 15239...|
| 27|  [105839362, 14179...|
| 29|  [25646344, 789631...|
| 61|  [62516620, 832532...|
| 63|  [107298055, 11094...|
|125|  [141797323, 52798...|
|140|  [25646344, 198263...|
|158|  [105839362, 25646...|
|298|  [77470563, 156415...|
|303|  [134346829, 13374...|
|389|  [25646344, 156415...|
|397|  [107298055, 52826...|
|410|  [25646344, 147936...|
|447|  [4403478, 6921930...|
|474|  [189933794, 55462...|
|533|  [49475876, 122226...|
|679|  [3778842, 4672831...|
|754|  [76887250, 136754...|
|796|  [141797323, 24313...|
+---+----------------------+
only showing top 20 rows



G:\soft\Anaconda\lib\site-packages\pyspark\sql\pandas\conversion.py:88: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.15.1 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
